### Heroes Of Pymoli Data Analysis
* Of the 576 active players, the vast majority are male (84.03%) There also exists, a smaller, but notable proportion of female players (14.06%)

* Our peak age demographic falls between 20-24 (44.79%) with secondary groups falling between 15-19 (18.58%) and 25-29 (13.37%).

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [177]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
player_purchase_data = purchase_data

## Player Count

* Display the total number of players


In [178]:
#Get the count of unique players
#Create a dicitonary with key value pair for total players
players = [{"Total Players": purchase_data["SN"].nunique()}]
total_players_df = pd.DataFrame(players)
total_players_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [179]:
# create a dictionary of unique items, average price, number of purchases and total revenue
display_dict  = [{"Number of Unique Items": purchase_data["Item ID"].nunique(),
                 "Average Price": purchase_data["Price"].mean(),
                 "Number of Purchases": purchase_data["Purchase ID"].count(),
                 "Total Revenue": purchase_data["Price"].sum()}]

#create DataFrame using the dictionary above
display_summary = pd.DataFrame(display_dict)
display(display_summary.style.format({"Average Price":"${:,.2f}","Total Revenue":"${:,.2f}"}))


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [180]:
#Get the unique players subset
player_df = purchase_data.drop_duplicates(subset = ["SN"])
#get the value counts foe each gender
gender_count_df =  pd.DataFrame(player_df.value_counts("Gender"), columns=["Total Count"])

#Calculate percentage of players
gender_count_df["Percentage of Players"] = round((100*gender_count_df["Total Count"])/player_df["SN"].nunique(),2)
display(gender_count_df)


,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [181]:
#Assign the purchase data to new variable
purchasing_analysis_df = purchase_data
#group the data frame by Gender
gender_group_df = purchasing_analysis_df.groupby("Gender")

#Calcualte count, sum and mean
analysis_display_df = gender_group_df.aggregate({"Purchase ID": "count","Price":["mean","sum"],"SN":"nunique" })

#Column names
column_names = ["Purchase_Count","Average_Purchase_Price","Total_Purchase_Value","Unique_Count"]
analysis_display_df.columns = column_names

#calculaye Average Total Purchase per Person
dummy_data= pd.DataFrame(round(analysis_display_df.Total_Purchase_Value/analysis_display_df.Unique_Count,2), columns = ["Avg_Total_Purchase_per_Person"])

#Assign the value of Average Total Purchase per Person to Unique_Count column
analysis_display_df["Unique_Count"]=dummy_data["Avg_Total_Purchase_per_Person"]
#rename the Unique_Count column to  Avg_Total_Purchase_per_Person
analysis_display_df.rename(columns={"Unique_Count": "Avg_Total_Purchase_per_Person"}, inplace=True)


#replace "_" in column names with " "
analysis_display_df.columns = display_df.columns.str.replace("_"," ")

#Format the columns to display in $ format
display(analysis_display_df.style.format({"Average Purchase Price":"${:,.2f}",
                              "Total Purchase Value":"${:,.2f}","Avg Total Purchase per Person":"${:,.2f}"}))

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [182]:
#Drop duplicate Players
unique_player_df = purchase_data.drop_duplicates(subset = ["SN"])

#Create age bins
age_bins=[0,9, 14, 19, 24, 29, 34, 39, 100]
age_labels = [" <10","10-14","15-19", "20-24","25-29", "30-34", "35-39", "40+"]
age_groups_df=pd.DataFrame(pd.cut(unique_player_df["Age"], age_bins, labels=age_labels))


#Get the count for each bin/age group
grouped_age_df = age_groups_df.groupby("Age")
grouped_age_df_count = pd.DataFrame(grouped_age_df["Age"].count())
grouped_age_df_count.rename(columns={"Age" : "Total Count"}, inplace=True)

# Calculating Percentage
grouped_age_df_count['Percentage of Players'] = round((grouped_age_df_count['Total Count'] * 100 )/ unique_player_df["SN"].count(),2)
grouped_age_df_count

,Total Count,Percentage of Players
Age,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [183]:
player_df1 = player_purchase_data

#Create age bins with labels
age_bins=[0,9, 14, 19, 24, 29, 34, 39, 100]
age_labels = [" <10","10-14","15-19", "20-24","25-29", "30-34", "35-39", "40+"]
age_groups_df=pd.DataFrame(pd.cut(player_df1["Age"], age_bins, labels=age_labels))
age_groups_df.columns = ["AgeGroup"]

#concatenate the player df1 and age group df
concat_df = pd.concat([age_groups_df ,player_df1], axis='columns', sort=False)

#Group the dataframe by AgeGroup
grouped_age_df = concat_df.groupby("AgeGroup")

#Calculate count, mean  and sum
display_df = grouped_age_df.aggregate({"Age": "count","Price":["mean","sum"],"SN":"nunique" })

#give the appropriate columns names
column_names = ["Purchase_Count","Average_Purchase_Price","Total_Purchase_Value","Unique_Count"]
display_df.columns = column_names

#Calculate Avg Total Purchase per Person and assign it to Unique_Count column
dummy_data= pd.DataFrame(round(display_df.Total_Purchase_Value/display_df.Unique_Count,2), columns = ["Avg_Total_Purchase_per_Person"])
display_df["Unique_Count"]=dummy_data["Avg_Total_Purchase_per_Person"]

#rename Unique_Count column to Avg_Total_Purchase_per_Person
display_df.rename(columns={"Unique_Count": "Avg_Total_Purchase_per_Person"}, inplace=True)

#Replace "_" in column names to " "
display_df.columns = display_df.columns.str.replace("_"," ")

#format to display in $
display(display_df.style.format({"Average Purchase Price":"${:,.2f}",
                              "Total Purchase Value":"${:,.2f}","Avg Total Purchase per Person":"${:,.2f}"}))


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
AgeGroup,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [184]:
SN_group_data = player_purchase_data.groupby("SN")
#calculate count, mean and sum
top_spender_df = SN_group_data.aggregate({"Purchase ID": "count","Price":["mean","sum"]})

#Give proper column names
top_spender_df.columns = ["Purchase Count","Average Purchase Price","Total Purchase Value"]
#sort by total purchase value and purchase count 
sorted_top_spender_df = top_spender_df.sort_values(by=["Total Purchase Value","Purchase Count"], ascending=[False,False]).head(5)

#format to display in $
display(sorted_top_spender_df.style.format({"Average Purchase Price":"${:,.2f}",
                              "Total Purchase Value":"${:,.2f}"}))

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [185]:
#Group by Item ID and Item Name
item_group_data = player_purchase_data.groupby(["Item ID","Item Name"])

#calculate count, unique and sum values
most_popular_items_df = item_group_data.aggregate({"Purchase ID": "count","Price":["unique","sum"]})


#give proper column names
most_popular_items_df.columns = ["Purchase Count","Item Price","Total Purchase Value"]

#Convert the object type pf column Item Price to float
most_popular_items_df["Item Price"] = most_popular_items_df["Item Price"].astype(float)

#sort by purchase count in descending order and show only first 5 records
sorted_most_popular_items_df= most_popular_items_df.sort_values(["Purchase Count"], ascending=False).head(5)

#Format columns to display in $
display(sorted_most_popular_items_df.style.format({"Item Price":"${:,.2f}",
                                "Total Purchase Value":"${:,.2f}"}))


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [186]:
#Sort the above dataframe by Total purchase value in descending order and show only first 5 records
most_profitable_items_df = most_popular_items_df.sort_values(by="Total Purchase Value", ascending=False).head(5)
#format to display in $
display(most_profitable_items_df.style.format({"Item Price":"${:,.2f}",
                                "Total Purchase Value":"${:,.2f}"}))

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
